In [10]:
import sklearn
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import Ridge, Lasso, LinearRegression, RidgeCV, LassoCV, LogisticRegression
from sklearn.metrics import r2_score, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, OneHotEncoder, LabelEncoder
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC, LinearSVC
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from tqdm import tqdm
import pandas as pd
import numpy as np

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
RANDOM_STATE = 42

In [13]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [231]:
np.random.seed(42)
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred = linear_model.predict(X_test)
r2_linear_model = r2_score(y_test, y_pred)
print(f'R2 метрика для линейной модели: {r2_linear_model}')

R2 метрика для линейной модели: 0.6687594935356318


3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [232]:
def degree_10():
    for i in range(10):
        yield 10e-5 * 10 ** i

### Ridge через GridSearchCV

In [233]:
parametrs = {'alpha' : [i for i in degree_10()]}
ridge_model = Ridge()
grid_ridge_model = GridSearchCV(ridge_model, parametrs, scoring='r2')
grid_ridge_model.fit(X_train, y_train)
y_pred_ridge = grid_ridge_model.predict(X_test)
r2_ridge_model = r2_score(y_test, y_pred_ridge)
print(f'R2 метрика для Ridge модели: {r2_ridge_model}')

R2 метрика для Ridge модели: 0.6687594145292219


### RidgeCV

In [234]:
ridgecv_model = RidgeCV(alphas=[i for i in degree_10()], scoring='r2')
ridgecv_model.fit(X_train, y_train)
y_pred_ridgecv = ridgecv_model.predict(X_test)
r2_ridgecv_model = r2_score(y_test, y_pred_ridgecv)
print(f'R2 метрика для RidgeCV модели: {r2_ridgecv_model}')

R2 метрика для RidgeCV модели: 0.6687510090373743


### Ridge через GridSearchCV

In [235]:
lasso_model = Lasso()
grid_lasso_model = GridSearchCV(lasso_model, parametrs, scoring='r2')
grid_lasso_model.fit(X_train, y_train)
y_pred_lasso = grid_lasso_model.predict(X_test)
r2_lasso_model = r2_score(y_test, y_pred_lasso)
print(f'R2 метрика для Lasso модели: {r2_lasso_model}')

R2 метрика для Lasso модели: 0.6687631534779594


### LassoCV

In [236]:
lassocv_model = LassoCV(alphas=[i for i in degree_10()])
lassocv_model.fit(X_train, y_train)
y_pred_lassocv = lassocv_model.predict(X_test)
r2_lassocv_model = r2_score(y_test, y_pred_lassocv)
print(f'R2 метрика для RidgeCV модели: {r2_lassocv_model}')

R2 метрика для RidgeCV модели: 0.6687631534779594


Результат одинаковый для всех моделей

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

### Ridge + StandardScaler

In [237]:
parametrs = {'ridge__alpha' : [i for i in degree_10()]}
pipe_ridge_ss = make_pipeline(StandardScaler(), Ridge())
grid_ridge_ss_model = GridSearchCV(pipe_ridge_ss, parametrs, scoring='r2')
grid_ridge_ss_model.fit(X_train, y_train)
y_pred_ridge_ss = grid_ridge_ss_model.best_estimator_.predict(X_test)
r2_grid_ridge_ss_model = r2_score(y_test, y_pred_ridge_ss)
print(f'R2 метрика для Lasso+StandardScaler модели: {r2_grid_ridge_ss_model}')

R2 метрика для Lasso+StandardScaler модели: 0.6684624359643561


In [238]:
grid_ridge_ss_model.best_params_

{'ridge__alpha': 1.0}

### Ridge + MinMaxScaler

In [239]:
parametrs = {'ridge__alpha' : [i for i in degree_10()]}
pipe_ridge_ss = make_pipeline(MinMaxScaler(), Ridge())
grid_ridge_ss_model = GridSearchCV(pipe_ridge_ss, parametrs, scoring='r2')
grid_ridge_ss_model.fit(X_train, y_train)
y_pred_ridge_ss = grid_ridge_ss_model.best_estimator_.predict(X_test)
r2_grid_ridge_ss_model = r2_score(y_test, y_pred_ridge_ss)
print(f'R2 метрика для Ridge+MinMaxScaler модели: {r2_grid_ridge_ss_model}')

R2 метрика для Ridge+MinMaxScaler модели: 0.6700309977617649


In [240]:
grid_ridge_ss_model.best_params_

{'ridge__alpha': 0.1}

### Lasso + StandardScaler

In [241]:
parametrs = {'lasso__alpha' : [i for i in degree_10()]}
pipe_lasso_ss = make_pipeline(StandardScaler(), Lasso())
grid_lasso_ss_model = GridSearchCV(pipe_lasso_ss, parametrs, scoring='r2')
grid_lasso_ss_model.fit(X_train, y_train)
y_pred_lasso_ss = grid_lasso_ss_model.best_estimator_.predict(X_test)
r2_grid_lasso_ss_model = r2_score(y_test, y_pred_lasso_ss)
print(f'R2 метрика для Lasso+MinMaxScaler модели: {r2_grid_lasso_ss_model}')

R2 метрика для Lasso+MinMaxScaler модели: 0.6687548978932025


In [242]:
grid_lasso_ss_model.best_params_

{'lasso__alpha': 0.0001}

### Lasso + MinMaxScaler

In [243]:
parametrs = {'lasso__alpha' : [i for i in degree_10()]}
pipe_lasso_mm = make_pipeline(MinMaxScaler(), Lasso())
grid_lasso_mm_model = GridSearchCV(pipe_lasso_mm, parametrs, scoring='r2')
grid_lasso_mm_model.fit(X_train, y_train)
y_pred_lasso_mm = grid_lasso_mm_model.best_estimator_.predict(X_test)
r2_grid_lasso_mm_model = r2_score(y_test, y_pred_lasso_mm)
print(f'R2 метрика для Lasso+MinMaxScaler модели: {r2_grid_lasso_mm_model}')

R2 метрика для Lasso+MinMaxScaler модели: 0.6687693392484069


In [244]:
grid_lasso_mm_model.best_params_

{'lasso__alpha': 0.0001}

Лучший результат у RidgeCV + MinMaxScaler = 0.6700309977617649

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

Посчитано в п.4

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [246]:
parametrs = {'ridge__alpha' : [i for i in degree_10()]}
pipe_ridge_ss = make_pipeline(StandardScaler(), PolynomialFeatures(2), Ridge())
grid_ridge_ss_model = GridSearchCV(pipe_ridge_ss, parametrs, scoring='r2')
grid_ridge_ss_model.fit(X_train, y_train)
y_pred_ridge_ss = grid_ridge_ss_model.best_estimator_.predict(X_test)
r2_grid_ridge_ss_model = r2_score(y_test, y_pred_ridge_ss)
print(f'R2 метрика для Lasso+StandardScaler+PolynomialFeatures модели: {r2_grid_ridge_ss_model}')

R2 метрика для Lasso+StandardScaler+PolynomialFeatures модели: 0.8180465877243648


In [247]:
parametrs = {'ridge__alpha' : [i for i in degree_10()]}
pipe_ridge_ss = make_pipeline(MinMaxScaler(), PolynomialFeatures(2), Ridge())
grid_ridge_ss_model = GridSearchCV(pipe_ridge_ss, parametrs, scoring='r2')
grid_ridge_ss_model.fit(X_train, y_train)
y_pred_ridge_ss = grid_ridge_ss_model.best_estimator_.predict(X_test)
r2_grid_ridge_ss_model = r2_score(y_test, y_pred_ridge_ss)
print(f'R2 метрика для Ridge+MinMaxScaler+PolynomialFeatures модели: {r2_grid_ridge_ss_model}')

R2 метрика для Ridge+MinMaxScaler+PolynomialFeatures модели: 0.8500630422288795


In [248]:
parametrs = {'lasso__alpha' : [i for i in degree_10()]}
pipe_lasso_ss = make_pipeline(StandardScaler(), PolynomialFeatures(2), Lasso())
grid_lasso_ss_model = GridSearchCV(pipe_lasso_ss, parametrs, scoring='r2')
grid_lasso_ss_model.fit(X_train, y_train)
y_pred_lasso_ss = grid_lasso_ss_model.best_estimator_.predict(X_test)
r2_grid_lasso_ss_model = r2_score(y_test, y_pred_lasso_ss)
print(f'R2 метрика для Lasso+MinMaxScaler+PolynomialFeatures модели: {r2_grid_lasso_ss_model}')

R2 метрика для Lasso+MinMaxScaler+PolynomialFeatures модели: 0.8122168137984359


In [249]:
parametrs = {'lasso__alpha' : [i for i in degree_10()]}
pipe_lasso_mm = make_pipeline(MinMaxScaler(), PolynomialFeatures(2), Lasso())
grid_lasso_mm_model = GridSearchCV(pipe_lasso_mm, parametrs, scoring='r2')
grid_lasso_mm_model.fit(X_train, y_train)
y_pred_lasso_mm = grid_lasso_mm_model.best_estimator_.predict(X_test)
r2_grid_lasso_mm_model = r2_score(y_test, y_pred_lasso_mm)
print(f'R2 метрика для Lasso+MinMaxScaler+PolynomialFeatures модели: {r2_grid_lasso_mm_model}');

R2 метрика для Lasso+MinMaxScaler+PolynomialFeatures модели: 0.8390581680518306


Результат значительно улучшился, лучший результат у Lasso+StandardScaler+PolynomialFeatures

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

In [225]:
res = {}
for estimator in tqdm([Lasso(), Ridge()]):
    for scaler in tqdm([MinMaxScaler(), StandardScaler()]):
        parametrs = {f'{str.lower(str(estimator))[:-2]}__alpha' : [i for i in degree_10()],
                     'polynomialfeatures__degree' : [2,3,4]}
        main_pipe = make_pipeline(scaler, PolynomialFeatures(), estimator)
        main_model = GridSearchCV(main_pipe, parametrs, scoring='r2')
        main_model.fit(X_train, y_train)
        y_pred_main = main_model.predict(X_test)
        res[f'{estimator}, {scaler}, {main_model.best_params_}'] = r2_score(y_test, y_pred_main)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:49<00:00, 24.88s/it]


In [226]:
res

{"Lasso(), MinMaxScaler(), {'lasso__alpha': 0.001, 'polynomialfeatures__degree': 3}": 0.8449734005131516,
 "Lasso(), StandardScaler(), {'lasso__alpha': 0.1, 'polynomialfeatures__degree': 2}": 0.8122168137984359,
 "Ridge(), MinMaxScaler(), {'polynomialfeatures__degree': 3, 'ridge__alpha': 0.1}": 0.8588479918651217,
 "Ridge(), StandardScaler(), {'polynomialfeatures__degree': 2, 'ridge__alpha': 10.0}": 0.8180465877243648}

Лучший результат Ridge(), MinMaxScaler(), {'polynomialfeatures__degree': 3, 'ridge__alpha': 0.1}": 0.8588479918651217

R2 мало видоизменялся до введения полинома

http://archive.ics.uci.edu/ml/datasets/Adult

In [36]:
# link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv('adult-all.csv', header=None)

In [37]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [38]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1].apply(lambda x: 0 if x =='>50K' else 1)

In [39]:
y.value_counts()

1    37155
0    11687
Name: 14, dtype: int64

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [40]:
X.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

10. Выберите колонки с числовыми и категориальными переменными.

In [41]:
categorical_features = [1,3,5,6,7,8,9,13]
numeric_features = [0,2,4,10,11,12]

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [42]:
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

numeric_transformer = MinMaxScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [45]:
accuracy_score(y_test, y_pred)

0.8491145460128979

In [46]:
(y_pred == y_test).sum() / len(y_pred)

0.8491145460128979

In [47]:
((clf.predict_proba(X_test)[:,1] > 0.5) == y_test).sum() / len(y_pred)

0.8491145460128979

In [48]:
f1_score(y_test, y_pred)

0.9039113428943938

13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [16]:
for estimator in tqdm([LogisticRegression(), SVC(), LinearSVC()]):
    clf = Pipeline(
        steps=[("preprocessor", preprocessor), ("classifier", estimator)]
    )
    acc = cross_val_score(clf, X, y, scoring='accuracy').mean()
    f1 = cross_val_score(clf, X, y, scoring='f1').mean()
    print(f'{estimator} : accuracy - {acc}, f1 - {f1}')

 33%|████████████████████████████                                                        | 1/3 [00:06<00:13,  6.74s/it]

LogisticRegression() : accuracy - 0.8511732716999377, f1 - 0.905011719035063


 67%|███████████████████████████████████████████████████████▎                           | 2/3 [14:55<08:45, 525.66s/it]

SVC() : accuracy - 0.839994419828589, f1 - 0.8986477856490989


100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [15:08<00:00, 302.85s/it]

LinearSVC() : accuracy - 0.8529135478362626, f1 - 0.9063223083526415


Лучший результат у LinearSVC() : accuracy - 0.8529135478362626, f1 - 0.9063223083526415

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [49]:
(X == '?').sum()

0        0
1     2799
2        0
3        0
4        0
5        0
6     2809
7        0
8        0
9        0
10       0
11       0
12       0
13     857
dtype: int64

In [50]:
(y == '?').sum()

0

In [51]:
data_columns = X.columns
imp_mean = SimpleImputer(missing_values='?', strategy='most_frequent')
new_data = pd.DataFrame(imp_mean.fit_transform(X), columns=data_columns)
new_data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

In [27]:
for estimator in tqdm([LogisticRegression(), SVC(), LinearSVC()]):
    clf = Pipeline(
        steps=[("preprocessor", preprocessor), ("classifier", estimator)]
    )
    acc = cross_val_score(clf, new_data, y, scoring='accuracy', n_jobs=-1).mean()
    f1 = cross_val_score(clf, new_data, y, scoring='f1', n_jobs=-1).mean()
    print(f'{estimator} : accuracy - {acc}, f1 - {f1}')

 33%|████████████████████████████                                                        | 1/3 [00:05<00:11,  5.62s/it]

LogisticRegression() : accuracy - 0.8505999501674596, f1 - 0.9047295620398798


 67%|███████████████████████████████████████████████████████▎                           | 2/3 [04:07<02:24, 144.70s/it]

SVC() : accuracy - 0.8395848963639935, f1 - 0.8985494397611463


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [04:12<00:00, 84.29s/it]

LinearSVC() : accuracy - 0.8508251523375897, f1 - 0.905141981677368


Результат не улучшился

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [50]:
data_droped_na = data.replace('?', np.NaN).dropna()
X_drop_na = data_droped_na.iloc[:,:-1]
y_drop_na = data_droped_na.iloc[:,-1].apply(lambda x: 0 if x =='>50K' else 1)

In [51]:
for estimator in tqdm([LogisticRegression(), SVC(), LinearSVC()]):
    clf = Pipeline(
        steps=[("preprocessor", preprocessor), ("classifier", estimator)]
    )
    acc = cross_val_score(clf, X_drop_na, y_drop_na, scoring='accuracy', n_jobs=-1).mean()
    f1 = cross_val_score(clf, X_drop_na, y_drop_na, scoring='f1', n_jobs=-1).mean()
    print(f'{estimator} : accuracy - {acc}, f1 - {f1}')

 33%|████████████████████████████                                                        | 1/3 [00:05<00:11,  5.92s/it]

LogisticRegression() : accuracy - 0.8468445746108516, f1 - 0.90114932100128


 67%|███████████████████████████████████████████████████████▎                           | 2/3 [03:27<02:00, 120.96s/it]

SVC() : accuracy - 0.8356995943179577, f1 - 0.8946665218391704


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [03:32<00:00, 70.70s/it]

LinearSVC() : accuracy - 0.8485030154158197, f1 - 0.9024033119235104


Результат не улучшился

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [52]:
for estimator in tqdm([RandomForestClassifier(), GradientBoostingClassifier()]):
    clf = Pipeline(
        steps=[("preprocessor", preprocessor), ("classifier", estimator)]
    )
    acc = cross_val_score(clf, X, y, scoring='accuracy', n_jobs=-1).mean()
    f1 = cross_val_score(clf, X, y, scoring='f1', n_jobs=-1).mean()
    print(f'{estimator} : accuracy - {acc}, f1 - {f1}')

 50%|██████████████████████████████████████████                                          | 1/2 [01:18<01:18, 78.52s/it]

RandomForestClassifier() : accuracy - 0.8530773228490498, f1 - 0.9049041758416918


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:31<00:00, 45.60s/it]

GradientBoostingClassifier() : accuracy - 0.8675731196536806, f1 - 0.9160399106672337


Результат улучшился, решающие деревья лучше решают данную задачу, особенно в ансамбле.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [58]:
for i in [0,1,2]:
    if i == 0:
        data_columns = X.columns
        imp_mean = SimpleImputer(missing_values='?', strategy='most_frequent')
        new_data = pd.DataFrame(imp_mean.fit_transform(X), columns=data_columns)
        for estimator in tqdm([RandomForestClassifier(), GradientBoostingClassifier()]):
            for numeric_scaler in [MinMaxScaler(), StandardScaler()]:
                preprocessor = ColumnTransformer(
                    transformers=[
                        ("num", numeric_scaler, numeric_features),
                        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
                    ]
                )
                clf = Pipeline(
                    steps=[("preprocessor", preprocessor), ("classifier", estimator)]
                )
                acc = cross_val_score(clf, new_data, y, scoring='accuracy', n_jobs=-1).mean()
                f1 = cross_val_score(clf, new_data, y, scoring='f1', n_jobs=-1).mean()
                test_for_res = f'{estimator}, {numeric_scaler}, {categorical_transformer}, imputer - most_frequent : accuracy - {acc}, f1 - {f1}'
                print(test_for_res)
    if i == 1:
        data_droped_na = data.replace('?', np.NaN).dropna()
        X_drop_na = data_droped_na.iloc[:,:-1]
        y_drop_na = data_droped_na.iloc[:,-1].apply(lambda x: 0 if x =='>50K' else 1)
        for estimator in tqdm([RandomForestClassifier(), GradientBoostingClassifier()]):
            for numeric_scaler in [MinMaxScaler(), StandardScaler()]:
                preprocessor = ColumnTransformer(
                    transformers=[
                        ("num", numeric_scaler, numeric_features),
                        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
                    ]
                )
                clf = Pipeline(
                    steps=[("preprocessor", preprocessor), ("classifier", estimator)]
                )
                acc = cross_val_score(clf, X_drop_na, y_drop_na, scoring='accuracy', n_jobs=-1).mean()
                f1 = cross_val_score(clf, X_drop_na, y_drop_na, scoring='f1', n_jobs=-1).mean()
                test_for_res = f'{estimator}, {numeric_scaler}, {categorical_transformer}, NaN dropped : accuracy - {acc}, f1 - {f1}'
                print(test_for_res)
    if i == 2:
        for estimator in tqdm([RandomForestClassifier(), GradientBoostingClassifier()]):
            for numeric_scaler in [MinMaxScaler(), StandardScaler()]:
                preprocessor = ColumnTransformer(
                    transformers=[
                        ("num", numeric_scaler, numeric_features),
                        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
                    ]
                )
                clf = Pipeline(
                    steps=[("preprocessor", preprocessor), ("classifier", estimator)]
                )
                acc = cross_val_score(clf, X, y, scoring='accuracy', n_jobs=-1).mean()
                f1 = cross_val_score(clf, X, y, scoring='f1', n_jobs=-1).mean()
                test_for_res = f'{estimator}, {numeric_scaler}, {categorical_transformer}, no change data : accuracy - {acc}, f1 - {f1}'
                print(test_for_res)

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

RandomForestClassifier(), MinMaxScaler(), OneHotEncoder(handle_unknown='ignore'), imputer - most_frequent : accuracy - 0.8519922159291594, f1 - 0.9044331346251274


 50%|█████████████████████████████████████████▌                                         | 1/2 [02:42<02:42, 162.97s/it]

RandomForestClassifier(), StandardScaler(), OneHotEncoder(handle_unknown='ignore'), imputer - most_frequent : accuracy - 0.8519511652841574, f1 - 0.9047362806564907
GradientBoostingClassifier(), MinMaxScaler(), OneHotEncoder(handle_unknown='ignore'), imputer - most_frequent : accuracy - 0.8663855852334714, f1 - 0.9153954075870411


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

...

GradientBoostingClassifier(), StandardScaler(), OneHotEncoder(handle_unknown='ignore'), NaN dropped : accuracy - 0.8629208373582479, f1 - 0.9122523878003121
RandomForestClassifier(), MinMaxScaler(), OneHotEncoder(handle_unknown='ignore'), no change data : accuracy - 0.8520125987292536, f1 - 0.9048407761956225


 50%|█████████████████████████████████████████▌                                         | 1/2 [02:48<02:48, 168.06s/it]

RandomForestClassifier(), StandardScaler(), OneHotEncoder(handle_unknown='ignore'), no change data : accuracy - 0.852974945650757, f1 - 0.9047524353566881
GradientBoostingClassifier(), MinMaxScaler(), OneHotEncoder(handle_unknown='ignore'), no change data : accuracy - 0.8675935946741558, f1 - 0.9160258070962349


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [03:15<00:00, 97.77s/it]

GradientBoostingClassifier(), StandardScaler(), OneHotEncoder(handle_unknown='ignore'), no change data : accuracy - 0.8675935946741558, f1 - 0.9160258070962349


Лучший результат: 
GradientBoostingClassifier(), StandardScaler(), OneHotEncoder(handle_unknown='ignore'), no change data : 

**accuracy - 0.8675935946741558, f1 - 0.9160258070962349**